In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import t as t_distribution

In [ ]:
import numpy as np 
from pandas import DataFrame
import seaborn as sns
%matplotlib inline

from copy import deepcopy
import matplotlib as plt

# Prep: Generate population of YT users by subgroup

## Population

### Census: Subgroup population in USA (https://www.statista.com/statistics/241488/population-of-the-us-by-sex-and-age/)
For each subgroup of the US population, get the number of people

In [ ]:
subgroup_pop = pd.read_csv('./subgroup_populations_census.txt', delimiter='\t')

In [ ]:
# Data preprocessing
# Convert values to millions

subgroup_pop.iloc[0,0] = '0 to 5'
subgroup_pop.iloc[-1,0] = '85 to 100'
subgroup_pop['age_low'] = subgroup_pop['Age in years'].apply(
    lambda x: int(x[:x.index(' to ')])
)
subgroup_pop = subgroup_pop.iloc[:,1:]
subgroup_pop[['Male','Female']] = subgroup_pop[['Male','Female']].apply(lambda x: x*1000000)
subgroup_pop.rename(columns={'Male':'male','Female':'female'},inplace=True)

subgroup_pop.head()

In [ ]:
# Create upper_age category (e.g. 50+ or 65+)
upper = 65
dict_upper = subgroup_pop[subgroup_pop['age_low']>=upper].agg(sum)
dict_upper['age_low'] = upper
df_upper = pd.DataFrame(dict_upper).T

# Remove 18 and below people
dict_18 = subgroup_pop[subgroup_pop['age_low']==15].agg(sum) * 2/5
dict_18['age_low'] = 18
df_18 = pd.DataFrame(dict_18).T

subgroup_pop = subgroup_pop[(subgroup_pop['age_low']>=18) & (subgroup_pop['age_low']<upper)]
subgroup_pop = pd.concat([subgroup_pop, df_18, df_upper])
subgroup_pop.sort_values('age_low',inplace=True)
subgroup_pop

In [ ]:
# Make age bins
bin_size = 20
bins_list = [18] + list(range(25,66,bin_size))
bins_list = list(zip(bins_list,bins_list[1:]+[100]))
bins = pd.IntervalIndex.from_tuples(bins_list,closed='left')

subgroup_pop['age_bucket'] = pd.cut(subgroup_pop['age_low'],bins=bins)
#subgroup_pop = subgroup_pop.groupby('age_bucket')['male','female'].agg('sum')
subgroup_pop = subgroup_pop.groupby('age_bucket')[['male','female']].agg('sum')

In [ ]:
# More preprocessing
subgroup_pop=subgroup_pop[['male','female']]
subgroup_pop = subgroup_pop.stack(-1)
subgroup_pop = subgroup_pop.rename('population')
subgroup_pop.index.set_names(['age_bucket','sex'],inplace=True)

In [ ]:
# Create dictionary mapping to unique categories
subgroup_df = subgroup_pop.to_frame().reset_index()
subgroup_df['cats_str'] = subgroup_df['age_bucket'].astype('str') + subgroup_df['sex']
cats = subgroup_df['cats_str'].unique()
cats_dict = dict(zip(cats, list(range(len(cats)))))

cats_dict

In [ ]:
subgroup_df['type'] = subgroup_df['cats_str'].apply(lambda x: cats_dict[x])
subgroup_df[['type','population']].to_csv('./post_stratification/categories.csv')

### PEW: Social media usage, by subgroup (https://www.pewresearch.org/internet/2021/04/07/social-media-use-in-2021/)
For each subgroup, get the number of people that use YouTube

In [ ]:
results = pd.read_csv('pew_core_survey_2021_results.csv')

In [ ]:
# Set index to sex and age, find ratios of YT usage

results['age_bucket'] = pd.cut(results['age'],bins=bins)
results['sex'].replace({1:'male',2:'female'},inplace=True)
results[['sex','age','age_bucket','web1e']]

usage_ratios = results.groupby(['age_bucket','sex'])['web1e'].agg(
    lambda x: int(x.value_counts()[1]) / len(x)
)

usage_ratios = usage_ratios.dropna()
usage_ratios.rename('usage',inplace=True)

usage_ratios.head()

### Social media ratios
Combine Census (subgroup_pop) and PEW (usage_ratios) data on subgroups to get the estimated number of Americans who use YouTube, by subgroup

In [ ]:
subgroup_pop.head(30)

In [ ]:
usage_ratios.head(20)

In [ ]:
# Find absolute number of people from each subgroup that use YT

# The values make sense
# For young folks there are more males bc the content is male-dominated. 
# For older folks it reflects more of the general us pop.

usage_pops = pd.DataFrame(usage_ratios).join(subgroup_pop)

yt_usage_abs = usage_pops['usage'] * usage_pops['population']
yt_usage_abs.rename('yt_usage_abs',inplace=True)

### [12/6] Map subgroups to a number category (or "type")

In [ ]:
cats_dict

In [ ]:
yt_usage_abs_df = yt_usage_abs.to_frame()
yt_usage_abs_df.reset_index(inplace=True)
yt_usage_abs_df['type'] = (
    yt_usage_abs_df['age_bucket'].astype('str') +
    yt_usage_abs_df['sex'].astype('str')
).apply(lambda x: cats_dict[x])

yt_usage_abs_2 = yt_usage_abs_df[['yt_usage_abs','type']].set_index('type')
yt_usage_abs_2

# Get responses from survey respondents

## Survey preprocessing

In [ ]:
buttons = ['delete','dislike','not_int','no_chan']

In [ ]:
qs = {
    'Q32': 'prolific_id',
    'Q16': 'experience',
    'Q92': 'attention_1',
    'Q95': 'attention_2',
    'Q97': 'attention_3'
}

button_qs = {
    'Q23': 'delete_aware',
    'Q24': 'delete_use_real',
    'Q26' : 'delete_use_hyp',
    'Q25' : 'delete_eff_real',
    'Q27' : 'delete_eff_hyp',
    
    'Q60': 'dislike_aware',
    'Q61': 'dislike_use_real',
    'Q62' : 'dislike_use_hyp',
    'Q63' : 'dislike_eff_real',
    'Q64' : 'dislike_eff_hyp',
    
    'Q67': 'not_int_aware',
    'Q68': 'not_int_use_real',
    'Q69' : 'not_int_use_hyp',
    'Q71' : 'not_int_eff_real',
    'Q72' : 'not_int_eff_hyp',
    
    'Q81': 'no_chan_aware',
    'Q82': 'no_chan_use_real',
    'Q83' : 'no_chan_use_hyp',
    'Q85' : 'no_chan_eff_real',
    'Q86' : 'no_chan_eff_hyp'    
}

qs.update(button_qs)

In [ ]:
survey = pd.read_csv('./yt_disinterest_survey_results.csv')
survey = survey.rename(
    columns=qs
)

In [ ]:
# rows of interest
# only get prolific_id-looking survey rows
survey.dropna(subset=['prolific_id'])
sample_pid = '610c250a4cf53941b3e3e55d'
survey.astype({'prolific_id':'object'})
survey['has_id']=survey['prolific_id'].apply(
    lambda x: len(str(x))==len(sample_pid)
)

survey=survey[survey['has_id']]
# column header rows
survey = survey.iloc[2:,:]

In [ ]:
# attention checks
survey = survey[
    (survey['attention_1'] == '2') &
    (survey['attention_2'] == 'Yes') &
    (survey['attention_3'].isin(['video','Video']))
]

In [ ]:
# columns of interest
survey = survey[list(qs.values())]
for att_idx in [1,2,3]:
    survey = survey.loc[:, survey.columns !='attention_{0}'.format(att_idx)]

# set index
survey.set_index('prolific_id',inplace=True)

# replace values
survey.replace({'Yes':1, 
                'Yes, sometimes': 1,
                'Yes, rarely': 1,
                'Yes, very frequently': 1,
                'Never': 0,
                'No':0, 
                'Not sure':pd.NA,
                '5- completely effective': 5,
                '1- not at all effective': 1
               }, inplace=True)

# change value types
for c in survey.columns:
    survey[c] = pd.to_numeric(survey[c])
survey = survey.astype('Int64')

In [ ]:
survey.shape

In [ ]:
# add more columns of interest
for b in buttons:
    survey['{0}_exp_aware'.format(b)] = (survey['experience'] * survey['{0}_aware'.format(b)]).replace({0:pd.NA})

    # [1/13] Hacky way of quickly getting aware given experience 
    survey['{0}_aware__exp'.format(b)] = survey['experience'].replace({0:pd.NA}) * survey['{0}_aware'.format(b)]
    
    survey['{0}_use__exp_aware'.format(b)] = survey['{0}_use_real'.format(b)] * survey['{0}_exp_aware'.format(b)]
    survey['{0}_eff__exp_aware_use'.format(b)] = survey['{0}_eff_real'.format(b)] * (survey['{0}_exp_aware'.format(b)] * survey['{0}_use_real'.format(b)]).replace({0:pd.NA})
    survey['{0}_eff__exp_aware_no_use'.format(b)] = survey['{0}_eff_hyp'.format(b)] * (survey['{0}_exp_aware'.format(b)] * survey['{0}_use_real'.format(b)]).replace({1:pd.NA, 0:1})
    survey['{0}_eff__exp_aware'.format(b)] = (survey['{0}_eff__exp_aware_use'.format(b)].replace({pd.NA:0}) + survey['{0}_eff__exp_aware_no_use'.format(b)].replace({pd.NA:0})).replace({0:pd.NA})


In [ ]:
#sanity check
assert(survey['no_chan_eff__exp_aware_no_use'].sum() == 
       survey[(survey['experience']==1)&(survey['no_chan_aware']==1)&(survey['no_chan_use_real']==0)]['no_chan_eff_hyp'].sum()
      )
       

In [ ]:
# Join with Prolific's demographics data

survey_dems=pd.read_csv('./prolific_demographics.csv')
survey_dems = survey_dems[['Participant id', 'Age', 'Sex']]
survey_dems.set_index('Participant id', inplace=True)
survey_dems.head()

survey = survey.join(survey_dems)

In [ ]:
# Change index to sex and age_bucket
survey['age_bucket'] = pd.cut(survey['Age'],bins=bins)
# Keep only binary genders
survey = survey[survey['Sex'].isin(['Male','Female'])]
survey['Sex'].replace({g:g.lower() for g in ['Male','Female']},inplace=True)

# Change each subgroup to a type number 
survey['cats_str'] = survey['age_bucket'].astype('str') + survey['Sex']
survey['type'] = survey['cats_str'].apply(lambda x: cats_dict[x])

In [ ]:
survey.head()

## Run survey diagnostics
How does our survey sample differ from estimated YT using population?

In [ ]:
# the frequency of each subgroup, 
#   for the sample that answered each question as well as the estimated YT using population    
bq_pop_list=[]

qs_list = []
for b in buttons:
    for q in ['aware','aware__exp','use__exp_aware','eff__exp_aware','eff__exp_aware_use','eff__exp_aware_no_use']:
        qs_list.append('{0}_{1}'.format(b,q))

for bq in qs_list:
    bq_pop_list.append(
        survey[~survey[bq].isna()]['type'].value_counts(normalize=True).rename(bq)
    )
bq_pop = pd.concat(bq_pop_list,axis=1).join(yt_usage_abs_2.apply(lambda x: x / x.sum()))


In [ ]:
# Get totals, rearrange rows and columns, na --> 0
bq_pop_sums = bq_pop.sum().rename('total')
bq_pop.replace({np.nan:0}, inplace=True)
bq_pop = bq_pop.loc[list(range(len(cats_dict.keys()))),:]
bq_pop.index = list(cats_dict.keys())

In [ ]:
# for heatmap
bq_pop_float = deepcopy(bq_pop)
bq_pop_float = bq_pop_float.loc[
    list(cats_dict.keys()),['yt_usage_abs']+qs_list
]

In [ ]:
fig, ax = plt.pyplot.subplots(figsize=(20,5))         # Sample figsize in inches

sns.heatmap(bq_pop_float, annot=True)

In [ ]:
bq_pop

## [12/6] Get stata file for post-stratification
Stata requires survey sample with (1) type (i.e. the category for post-strat) and (2) the number of the type in the population.

In [ ]:
survey_cats = survey.join(yt_usage_abs_2, on='type')

In [ ]:
survey_cats.head()

In [ ]:
survey_cats['yt_usage_abs'] = survey_cats['yt_usage_abs'].astype('int')
survey_cats['n_pop'] = [int(yt_usage_abs_2.sum()[0]) for i in range(survey_cats.shape[0])]
survey_cats.rename(columns={'yt_usage_abs':'n_type'},inplace=True)

In [ ]:
survey_cats.to_stata('./post_stratification/sex_age_{0}.dta'.format(bin_size), write_index=False)

# ^ This gives a warning, which I've checked is fine in Stata
# Basically, it's confirmed that NA is registered in Stata and not used in mean calculations


### [12/10] Get counts 

In [ ]:
survey_n=survey_cats[qs_list].apply(lambda x: len(x) - x.isna().sum()).rename('n')
survey_n

In [ ]:
survey_n.to_csv('./survey_sample_size.csv')

In [ ]:
survey

In [ ]:
# Percentage of a series with 1 (discounting NA's)
def ratio_yes(series):
    num_1 = (series==1).sum()
    num_0 = (series==0).sum()
    
    if num_0 + num_1 == 0:            
        return pd.NA
    elif num_0 == 0:
        return 1
    else:
        return num_1 / (num_0 + num_1)

# Don't use provided std. Instead, use Bernouli std calculation
def std(series):
    # return series.std()
    r = ratio_yes(series)
    return np.sqrt(r * (1-r))

In [ ]:
survey_buckets = survey.groupby(['age_bucket','Sex']).agg([ratio_yes, 'count'])
survey_buckets.index.rename(['age_bucket','sex'],inplace=True)

In [ ]:
survey_buckets.head(20)

In [ ]:
test = survey_buckets['dislike_aware'].join(yt_usage_abs.rename('N'))
test['N_r'] = test['N']/sum(test['N'])

In [ ]:
test

In [ ]:
test.dropna()

In [ ]:
test['ratio_yes'].replace({pd.NA:0})

In [ ]:
sum(test['ratio_yes'].replace({pd.NA:0}) * test['N_r'])

In [ ]:
results_list = []

# Perform post-stratification
for b in buttons:
    temp_df = survey_buckets[b+'_aware'].join(yt_usage_abs.rename('N')).dropna()
    # Printing confirms that for every age bucket of survey respondents, we know their absolute number
    # In retrospect, let's cutoff at 18 below and do 65+ category for above
    #print(temp_df)
    
    temp_df.rename(columns={
        'ratio_yes': 'y_bar',
    }, inplace=True)
    
    # Calculate population ratios 
    temp_df['N_ratio'] = temp_df['N'] / sum(temp_df['N'])
    
    #print(temp_df)
    # make sure ratios add to 1
    assert(abs(1-temp_df['N_ratio'].sum()) < 0.00001)
    new_usage_ratio = sum(temp_df['y_bar'].replace({pd.NA:0}) * temp_df['N_ratio'])
    old_usage_ratio = ratio_yes(survey[b+'_aware'])

    print('{0}: {1:.2f}% ({2:.2f}%)'.format(b+'_aware',new_usage_ratio*100,old_usage_ratio*100))
    
    results_list.append({
        'button': b,
        'y_ps': new_usage_ratio,
        'y_old': old_usage_ratio,
    })

In [ ]:
results_df = pd.DataFrame(results_list)
results_df[['button','y_ps','y_old']].head()